# Cleaning scripts

In [1]:
import pandas as pd

In [2]:
years = list(range(1950, 2100+1, 5))

In [3]:
popshare = pd.read_csv('raw/WPP2022_PopulationByAge5GroupSex_Percentage_Medium.csv')
popshare = popshare.loc[popshare['Time'].isin(years)].reset_index(drop=True)

/var/folders/cv/xn1my8q111q1ngz5g62wkt_r0000gn/T/ipykernel_1127/1698792024.py:1: DtypeWarning: Columns (2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  popshare = pd.read_csv('raw/WPP2022_PopulationByAge5GroupSex_Percentage_Medium.csv')


In [4]:
pop = pd.read_csv('raw/WPP2022_PopulationByAge5GroupSex_Medium.csv')
pop = pop.loc[pop['Time'].isin(years)].reset_index(drop=True)

/var/folders/cv/xn1my8q111q1ngz5g62wkt_r0000gn/T/ipykernel_1127/1828745933.py:1: DtypeWarning: Columns (2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  pop = pd.read_csv('raw/WPP2022_PopulationByAge5GroupSex_Medium.csv')


In [5]:
median_age = pd.read_csv('raw/WPP2022_Demographic_Indicators_Medium.csv')
median_age = median_age.loc[median_age['Time'].isin(years)].reset_index(drop=True)
median_age = median_age[(median_age['LocTypeID'] == 1) | (median_age['LocTypeID'] == 4)]
median_age = median_age[['LocID', 'Location', 'Time', 'MedianAgePop']]

/var/folders/cv/xn1my8q111q1ngz5g62wkt_r0000gn/T/ipykernel_1127/1733417568.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  median_age = pd.read_csv('raw/WPP2022_Demographic_Indicators_Medium.csv')


In [6]:
median_age.to_csv('median_age.csv', index=False)

## Wrangle into relational database

In [7]:
age_groups = popshare[['AgeGrp']]
age_groups = age_groups.drop_duplicates(ignore_index=True)
age_groups.to_csv('age_groups.csv', index=False)

In [8]:
age_groups

,AgeGrp
0,0-4
1,5-9
2,10-14
3,15-19
4,20-24
5,25-29
6,30-34
7,35-39
8,40-44
9,45-49


In [9]:
countries = popshare[(popshare['LocTypeID'] == 1) | (popshare['LocTypeID'] == 4)]
countries = countries[['LocID', 'ParentID', 'Location']]
countries = countries.drop_duplicates(subset=['LocID'], ignore_index=True)
countries = countries.dropna(ignore_index=True)
countries

,LocID,ParentID,Location
0,900,0,World
1,108,910,Burundi
2,174,910,Comoros
3,262,910,Djibouti
4,232,910,Eritrea
...,...,...,...
232,882,957,Samoa
233,772,957,Tokelau
234,776,957,Tonga
235,798,957,Tuvalu


In [10]:
countries.to_csv('countries.csv', index=False)

In [11]:
continents = popshare[popshare['LocTypeID'] == 2]
continents = continents[['LocID', 'Location']]
continents = continents.drop_duplicates(subset=['LocID'], ignore_index=True)
continents = continents.dropna(ignore_index=True)
continents

,LocID,Location
0,903,Africa
1,935,Asia
2,908,Europe
3,904,Latin America and the Caribbean
4,905,Northern America
5,909,Oceania


In [12]:
regions = popshare[popshare['LocTypeID'] == 3]
regions = regions[['LocID', 'ParentID', 'Location']]
regions = regions.drop_duplicates(subset=['LocID'], ignore_index=True)
regions = regions.dropna(ignore_index=True)
regions

,LocID,ParentID,Location
0,910,903,Eastern Africa
1,911,903,Middle Africa
2,912,903,Northern Africa
3,913,903,Southern Africa
4,914,903,Western Africa
5,5500,935,Central Asia
6,906,935,Eastern Asia
7,5501,935,Southern Asia
8,920,935,South-Eastern Asia
9,922,935,Western Asia


## Consolidate and export

In [13]:
pop_world = pop[pop['Location'] == 'World']
pop_world = pop_world[
    ['LocID', 'Time', 'AgeGrp', 'PopMale', 'PopFemale']
].reset_index(drop=True)

popshare_world = popshare[popshare['Location'] == 'World']
popshare_world = popshare_world[
    ['LocID', 'Location', 'ParentID', 'Time', 'AgeGrp', 'PopMale', 'PopFemale']
].reset_index(drop=True)
popshare_world = popshare_world.rename(columns={'PopMale': 'PopShareMale', 'PopFemale': 'PopShareFemale'})

In [14]:
pop_countries = pop[pop['LocTypeID'] == 4]
pop_countries = pop_countries[
    ['LocID', 'Time', 'AgeGrp', 'PopMale', 'PopFemale']
].reset_index(drop=True)

popshare_countries = popshare[popshare['LocTypeID'] == 4]
popshare_countries = popshare_countries[
    ['LocID', 'Location', 'ParentID', 'Time', 'AgeGrp', 'PopMale', 'PopFemale']
].reset_index(drop=True)
popshare_countries = popshare_countries.rename(columns={'PopMale': 'PopShareMale', 'PopFemale': 'PopShareFemale'})

In [15]:
pop_countries = pd.concat([pop_world, pop_countries], ignore_index=True)
popshare_countries = pd.concat([popshare_world, popshare_countries], ignore_index=True)

In [16]:
popshare_countries

,LocID,Location,ParentID,Time,AgeGrp,PopShareMale,PopShareFemale
0,900,World,0,1950,0-4,6.988,6.691
1,900,World,0,1950,5-9,5.478,5.234
2,900,World,0,1950,10-14,5.286,5.040
3,900,World,0,1950,15-19,4.822,4.668
4,900,World,0,1950,20-24,4.442,4.365
...,...,...,...,...,...,...,...
154282,876,Wallis and Futuna Islands,957,2100,80-84,2.628,2.689
154283,876,Wallis and Futuna Islands,957,2100,85-89,2.426,2.729
154284,876,Wallis and Futuna Islands,957,2100,90-94,1.728,2.012
154285,876,Wallis and Futuna Islands,957,2100,95-99,0.728,1.071


In [17]:
pyramid = pd.merge(
    popshare_countries, 
    pop_countries, 
    on=['LocID', 'Time', 'AgeGrp'], 
    how='left'
).reset_index(drop=True)

Add columns to show Male/Female surplus.

In [18]:
pyramid[['SurplusShareMale', 'SurplusShareFemale', 'SurplusMale', 'SurplusFemale']] = pyramid.apply(
    lambda row: [
        row['PopShareMale'] - row['PopShareFemale'] if row['PopShareMale'] > row['PopShareFemale'] else 0, 
        row['PopShareFemale'] - row['PopShareMale'] if row['PopShareFemale'] > row['PopShareMale'] else 0,
        row['PopMale'] - row['PopFemale'] if row['PopMale'] > row['PopFemale'] else 0, 
        row['PopFemale'] - row['PopMale'] if row['PopFemale'] > row['PopMale'] else 0
    ], 
    axis=1, 
    result_type='expand'
)
pyramid

,LocID,Location,ParentID,Time,AgeGrp,PopShareMale,PopShareFemale,PopMale,PopFemale,SurplusShareMale,SurplusShareFemale,SurplusMale,SurplusFemale
0,900,World,0,1950,0-4,6.988,6.691,174647.391,167229.405,0.297,0.000,7417.986,0.000
1,900,World,0,1950,5-9,5.478,5.234,136914.008,130816.575,0.244,0.000,6097.433,0.000
2,900,World,0,1950,10-14,5.286,5.040,132112.875,125967.993,0.246,0.000,6144.882,0.000
3,900,World,0,1950,15-19,4.822,4.668,120512.860,116660.508,0.154,0.000,3852.352,0.000
4,900,World,0,1950,20-24,4.442,4.365,111019.768,109095.897,0.077,0.000,1923.871,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154282,876,Wallis and Futuna Islands,957,2100,80-84,2.628,2.689,0.260,0.266,0.000,0.061,0.000,0.006
154283,876,Wallis and Futuna Islands,957,2100,85-89,2.426,2.729,0.240,0.270,0.000,0.303,0.000,0.030
154284,876,Wallis and Futuna Islands,957,2100,90-94,1.728,2.012,0.171,0.199,0.000,0.284,0.000,0.028
154285,876,Wallis and Futuna Islands,957,2100,95-99,0.728,1.071,0.072,0.106,0.000,0.343,0.000,0.034


In [19]:
pyramid.to_csv('pyramid.csv', index=False)

## Fertility

In [20]:
fertility = pd.read_csv('raw/WPP2022_Demographic_Indicators_Medium.csv')
fertility = fertility.loc[fertility['Time'].isin(list(range(1950, 2022+1)))].reset_index(drop=True)

fertility_regions = fertility[(fertility['LocTypeID'] == 1) | (fertility['LocTypeID'] == 2)]
fertility_countries = fertility[fertility['LocTypeID'] == 4]

fertility_regions = fertility_regions[['LocID', 'Location', 'Time', 'TFR']]
fertility_countries = fertility_countries[['LocID', 'Location', 'Time', 'TFR']]

/var/folders/cv/xn1my8q111q1ngz5g62wkt_r0000gn/T/ipykernel_1127/3541316337.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  fertility = pd.read_csv('raw/WPP2022_Demographic_Indicators_Medium.csv')


In [21]:
fertility_regions.to_csv('fertility_regions.csv', index=False)
fertility_countries.to_csv('fertility_countries.csv', index=False)